In [1]:
import ee
import geemap
import os
import math
import geopandas as gpd
# 用户验证
# ee.Authenticate()
# 初始化Google Earth Engine
ee.Initialize()

In [2]:
# 切分区域
def split_region(region, col_size, row_size):
    bounds = region.getInfo()['coordinates'][0]
    original_minx, original_miny = bounds[0]
    original_maxx, original_maxy = bounds[2]
    
    x_len = original_maxx - original_minx
    y_len = original_maxy - original_miny
    
    # 向上取整
    cols = math.ceil(x_len / col_size)
    rows = math.ceil(y_len / row_size)
    
    rectangles = []
    for i in range(cols):
        current_minx = original_minx + i * col_size
        current_maxx = current_minx + col_size
        if current_maxx > original_maxx:
            current_maxx = original_maxx
        for j in range(rows):
            current_miny = original_miny + j * row_size
            current_maxy = current_miny + row_size
            if current_maxy > original_maxy:
                current_maxy = original_maxy
            rect = ee.Geometry.Rectangle([current_minx, current_miny, current_maxx, current_maxy])
            rectangles.append(rect)
    return rectangles

In [27]:
# 读取本地Shapefile 得到处理区域
shapefile_path = r"F:\ArcgisData\shp_polygon\KY500.shp"
gdf = gpd.read_file(shapefile_path)


bounds = gdf['geometry'][0].bounds
bbox = {
    'MinX': bounds[0],
    'MinY': bounds[1],
    'MaxX': bounds[2],
    'MaxY': bounds[3]
}

# 获取第一个几何对象
geometry = gdf.geometry.iloc[0]
# 2. 将 Earth Engine 几何对象转换为 Feature 对象
ee_geometry = ee.Geometry(geometry.__geo_interface__)
# 3. 将 Earth Engine 几何对象转换为 Feature 对象
feature = ee.Feature(ee_geometry)

# 4. 将 Feature 对象放入 FeatureCollection 中
feature_collection = ee.FeatureCollection([feature])

# 获取 FeatureCollection 的边界几何对象
bounded_geometry = feature_collection.geometry().bounds()


In [ ]:


# 读取本地Shapefile得到处理区域
shapefile_path = r"F:\ArcgisData\shp_polygon\DY500.shp"
gdf = gpd.read_file(shapefile_path)

# 获取第一个几何对象
geometry = gdf.geometry.iloc[0]

# 将 GeoPandas 几何对象转换为 Earth Engine 几何对象
ee_geometry = geemap.geopandas_to_ee(gdf, geodesic=False)

# 获取 FeatureCollection 的边界几何对象
bounded_geometry = ee_geometry.geometry().bounds()

# 定义时间范围和地区
start_date = '2020-02-01'
end_date = '2022-03-31'

# 筛选Landsat数据集
# dataset = ee.ImageCollection('LANDSAT/LC09/C02/T2_L2') \
#             .filterDate(start_date, end_date) \
#             .filterBounds(bounded_geometry)
dataset = ee.ImageCollection('LANDSAT/LC09/C02/T2_L2')

# 应用缩放因子并计算地表温度
def apply_scale_factors(image):
    # 计算地表温度，这里使用ST_B10波段
    temperature = image.select('ST_B10').multiply(0.00341802).add(149.0).rename('Temperature')
    return image.addBands(temperature)

# 应用函数
dataset = dataset.map(apply_scale_factors)

# 设置地表温度的可视化参数
visualization = {
    'bands': ['Temperature'],
    'min': 270,  # 设置最小值
    'max': 310,  # 设置最大值
    'palette': ['blue', 'green', 'red']  # 温度范围的颜色映射
}

# 可视化地表温度
Map = geemap.Map()
Map.centerObject(bounded_geometry, 8)  # 中心定位到区域
Map.addLayer(dataset.mean(), visualization, 'Temperature')
Map.addLayerControl()  # 添加图层控制
Map  # 显示地图
